# 201521466 김남현


### GridSearchCV

In [11]:
import csv
contents = []
labels = []
with open('petition_sample.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            contents.append(row[0]) #청원내용을 contents 리스트에 추가
            labels.append(row[1]) #청원 category를 labels 리스트에 추가
            

In [12]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

print("Parameter grid:\n{}".format(param_grid))
# 매개변수로 사용할 분류모형, parameter 집합과 cross-validation의 k 등을 지정
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)

Parameter grid:
{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}


In [18]:
X_train, X_test, y_train, y_test = train_test_split(contents, labels, random_state=0)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = twitter_tag.nouns, min_df = 3) #min_df 값 3으로 설정해보았다.

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

In [25]:
grid_search.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', grid_search.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', grid_search.score(X_test_tfidf, y_test))

Train score 0.9806666666666667
Test score 0.796


트레인셋이 98퍼센트, 테스트셋이 79.6퍼센트의 예측률을 각각 보인다.

In [42]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best score: {:.2f}".format(grid_search.best_score_)) 
# 사용한 파라미터 값 중에서 최선의 파라미터와 그에 대한 결과값을 나타낸다.

Best parameters: {'C': 10, 'gamma': 1}
Best score: 0.79


0.79가 최고 훈련정확도이다.

### Ridge regression

In [26]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.941
Test set score: 0.797


트레인셋이 94.1퍼센트, 테스트셋이 79.7퍼센트의 예측률을 각각 보인다.

### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습

print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

Train set score: 0.893
Test set score: 0.791


트레인셋이 89.3퍼센트, 테스트셋이 79.1퍼센트의 예측률을 각각 보인다.

### Naive Bayse Classifier

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 8000).fit(X_train) #max_feature를 제한하여 학습

X_train_cv = cv.transform(X_train) # train set을 변환
X_test_cv = cv.transform(X_test) # test set을 변환

In [38]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # Naive Bayse Classifier 분류기 선언
NB_clf.fit(X_train_cv, y_train)

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) 
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set score: 0.826
Test set score: 0.704


트레인셋이 82.6퍼센트, 테스트셋이 70.4퍼센트의 예측률을 각각 보인다. max_feature 값이 올라갈수록 더 좋은 예측률을 보인다.

### 유사 텍스트 검색 및 추천

In [39]:
from sklearn.metrics.pairwise import cosine_similarity


X = cv.fit_transform(contents) # 대상 텍스트를 counter vectorize.

new_petition = "공단이 꼭 북한에 있어야 합니까" # 해당 문장과 가장 유사한 부분을 찾을 계획.
new_vec = cv.transform([new_petition]) #문서를 count vector로 변환

sim_result = cosine_similarity(new_vec, X) # X와 기존 값들과의 similarity 계산

In [40]:
import numpy as np
np.argmax(sim_result[0]) 
# 가장 유사한 부분의 첫 번째 결과 위치 출력...인덱스

1033

In [41]:
print(contents[1033])  # 1033번째 텍스트 출력.

더이상의 피해자가 없도록 해주세요\n어린 나이더라도 엄격한 처벌이 있어야 한다고 생각합니다. 저들은 분명한 살인미수입니다.\n말도안되는 나라 말고 나라다운 나라 ..\n간절히 부탁드립니다


뭔가 잘못되었음을 느낀다. 완전 다른 내용이다.